In [1]:
#import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from datetime import timedelta  
import pandas as pd
import json
import numpy as np
import requests 
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Stations
import csv

In [2]:
from meteostat import Stations

# Get nearby weather stations
stations = Stations()
stations = stations.nearby(44.8620, -93.5323)
station = stations.fetch()
station

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
KFCM0,Minneapolis / Eden Prairie,US,MN,<NA>,KFCM,44.8275,-93.4586,276.0,America/Chicago,2006-01-01,2022-11-06,2006-01-02,2022-04-24,2006-01-01,2022-01-01,6.962625e+03
KMIC0,Minneapolis / Brooklyn Center,US,MN,<NA>,KMIC,45.0620,-93.3539,265.0,America/Chicago,2006-01-01,2022-11-06,2006-01-01,2022-04-24,2006-01-01,2021-01-01,2.629809e+04
KLVN0,Minneapolis / Lakeville,US,MN,<NA>,KLVN,44.6279,-93.2281,293.0,America/Chicago,2006-01-01,2022-11-06,2006-01-01,2022-04-24,2007-01-01,2022-01-01,3.542274e+04
72658,Minneapolis-St Paul International,US,MN,72658,KMSP,44.8667,-93.0500,256.0,America/Chicago,1945-01-01,2022-11-06,1938-04-09,2022-10-29,1938-01-01,2022-01-01,3.801486e+04
KSTP0,Saint Paul,US,MN,<NA>,KSTP,44.9346,-93.0603,215.0,America/Chicago,1983-11-03,2022-11-06,1991-12-16,2022-04-24,2005-01-01,2022-01-01,3.804391e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89592,Mirnyj,AQ,<NA>,89592,<NA>,-66.5000,93.0000,30.0,Antarctica/Mawson,NaT,NaT,1973-01-01,2022-10-29,1957-01-01,2021-01-01,1.757754e+07
61998,Port-Aux-Francais Iles Kerguelen,TF,KG,61998,<NA>,-49.3500,70.2500,29.0,Europe/Paris,NaT,NaT,1973-01-16,2022-11-01,1954-01-01,2022-01-01,1.869344e+07
95997,Atlas Cove (Heard Island),NF,<NA>,95997,<NA>,-53.0189,73.3917,3.0,Pacific/Norfolk,NaT,NaT,1998-03-01,2012-12-20,1999-01-01,2012-01-01,1.870174e+07


In [5]:
from datetime import datetime
from meteostat import Hourly

#number of years that will be captured in the data request
years_back = 10
days_back = years_back*365
now = datetime.utcnow().replace(microsecond=0, second=0, minute=0)
start = now - timedelta(days=days_back)
now_string = now.strftime("%Y-%m-%d")
start_string = start.strftime("%Y-%m-%d")


# Get hourly data from MSP airport (site number 72648)
data = Hourly('72658', start, now)
data = data.fetch()
data = data.reset_index()
data['Date'] = data['time'].dt.date

data = (data.set_index("time")
                   .tz_localize("utc")
                   .tz_convert("US/Central")
                   .reset_index()
          )

data

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,Date
0,2015-12-31 18:00:00-06:00,-8.3,-12.3,73.0,NaN,NaN,250.0,14.8,NaN,1023.3,NaN,NaN,2016-01-01
1,2015-12-31 19:00:00-06:00,-8.3,-11.8,76.0,NaN,NaN,240.0,14.8,NaN,1023.3,NaN,NaN,2016-01-01
2,2015-12-31 20:00:00-06:00,-7.8,-11.1,77.0,NaN,NaN,240.0,11.2,NaN,1022.9,NaN,NaN,2016-01-01
3,2015-12-31 21:00:00-06:00,-7.8,-11.1,77.0,NaN,NaN,250.0,14.8,NaN,1022.7,NaN,NaN,2016-01-01
4,2015-12-31 22:00:00-06:00,-7.2,-10.5,77.0,0.0,NaN,270.0,14.8,NaN,1022.8,NaN,NaN,2016-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87594,2015-12-31 13:00:00-06:00,-10.0,-13.9,73.0,0.0,NaN,220.0,16.6,NaN,1023.5,NaN,NaN,2015-12-31
87595,2015-12-31 14:00:00-06:00,-8.9,-12.9,73.0,NaN,NaN,230.0,11.2,NaN,1023.1,NaN,14.0,2015-12-31
87596,2015-12-31 15:00:00-06:00,-8.9,-12.9,73.0,NaN,NaN,250.0,16.6,NaN,1023.0,NaN,14.0,2015-12-31
87597,2015-12-31 16:00:00-06:00,-8.3,-12.8,70.0,NaN,NaN,230.0,18.4,NaN,1023.0,NaN,NaN,2015-12-31


In [ ]:
export_df.to_csv(f"weather_msp.csv")

In [6]:
station.to_csv(f"station.csv")